In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic Claude, Google Gemini

openai = OpenAI()
claude = anthropic.Anthropic()
gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-1.5-flash"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way. \
Generate one sentence at a time"

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting. \
Generate one sentence at a time"

gemini_system = "You are a neutral chatbot with no emotional bias. \
Generate one sentence at a time"

In [ ]:
def combine_msg(model1, msg1, model2, msg2):
    return model1 + " said: " + msg1 + "\n\n Then " + model2 + " said: " + msg1 + "."

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt_msg, claude_msg, gemini_msg in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt_msg})
        messages.append({"role": "user", "content": combine_msg("Claude", claude_msg, "Gemini", gemini_msg)})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
def call_claude():
    messages = []
    messages.append({"role": "user", "content": "GPT said: " + gpt_messages[0]})
    # the length of gpt_messages: n + 1
    # the length of claude_messages and gemini_messages: n
    for i in range(len(claude_messages)): 
        claude_msg = claude_messages[i]
        gemini_msg = gemini_messages[i]
        gpt_msg = gpt_messages[i + 1]
        messages.append({"role": "assistant", "content": claude_msg})
        messages.append({"role": "user", "content": combine_msg("Gemini", gemini_msg, "GPT", gpt_msg)})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
def call_gemini():
    messages = []
    messages.append({"role": "system", "content": gemini_system})
    messages.append({"role": "user", "content": combine_msg("GPT", gpt_messages[0], "Claude", claude_messages[0])})
    # the length of gpt_messages and claude_messages: n + 1
    # the length of gemini_messages: n
    for i in range(len(gemini_messages)): 
        gemini_msg = gemini_messages[i]
        gpt_msg = gpt_messages[i + 1]
        claude_msg = claude_messages[i + 1]
        messages.append({"role": "assistant", "content": gemini_msg})
        messages.append({"role": "user", "content": combine_msg("GPT", gpt_msg, "Claude", claude_msg)})
    response = gemini_via_openai_client.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
gpt_messages = ["Hi there."]
claude_messages = ["Hi."]
gemini_messages = ["Hi."]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)